- **What is [Chatbot](https://data-flair.training/blogs/python-chatbot-project/)?**
    - _A chatbot is an intelligent piece of software that is capable of communicating and performing actions similar to a human. Chatbots are used a lot in customer interaction, marketing on social network sites and instantly messaging the client. There are two basic types of chatbot models based on how they are built; Retrieval based and Generative based models._
    
1. Retrieval based Chatbots
    - A retrieval-based chatbot uses predefined input patterns and responses. It then uses some type of heuristic approach to select the appropriate response. It is widely used in the industry to make goal-oriented chatbots where we can customize the tone and flow of the chatbot to drive our customers with the best experience.


2. Generative based Chatbots
    - Generative models are not based on some predefined responses.
    - They are based on seq 2 seq neural networks. It is the same idea as machine translation. In machine translation, we translate the source code from one language to another language but here, we are going to transform input into an output. It needs a large amount of data and it is based on Deep Neural networks.
    
**About the Python Project – Chatbot**
    - In this Python project with source code, we are going to build a chatbot using deep learning techniques. The chatbot will be trained on the dataset which contains categories (intents), pattern and responses. We use a special recurrent neural network (LSTM) to classify which category the user’s message belongs to and then we will give a random response from the list of responses.

_Let’s create a retrieval based chatbot using NLTK, Keras, Python, etc._

**Download Chatbot Code & Dataset**
     - The dataset we will be using is ‘intents.json’. This is a JSON file that contains the patterns we need to find and the responses we want to return to the user.

**Please download python chatbot code & dataset from the following link:** [Python Chatbot Code & Dataset](https://data-flair.s3.ap-south-1.amazonaws.com/Data-Science-Code/chatbot-python-project-data-codes.zip)

**How to Make Chatbot in Python?**

|Filename|Description|
|----:|:----|
| Intents.json|The data file which has predefined patterns and responses.|
| train_chatbot.py|In this Python file, we wrote a script to build the model and train our chatbot.|
| Words.pkl|This is a pickle file in which we store the words Python object that contains a list of our vocabulary.|
| Classes.pkl|The classes pickle file contains the list of categories.|
| Chatbot_model.h5|This is the trained model that contains information about the model and has weights of the neurons.|
| Chatgui.py|This is the Python script in which we implemented GUI for our chatbot. Users can easily interact with the bot.|

**Here are the 5 steps to create a chatbot in Python from scratch:**

* Import and load the data file
* Preprocess data
* Create training and testing data
* Build the model
* Predict the response

1. Import and load the data file

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('chatbot-python-project-data-codes/intents.json').read()
intents = json.loads(data_file)

In [25]:
import pandas as pd

pd.DataFrame(intents['intents'])

,tag,patterns,responses,context
0,greeting,"[Hi there, How are you, Is anyone there?, Hey,...","[Hello, thanks for asking, Good to see you aga...",[]
1,goodbye,"[Bye, See you later, Goodbye, Nice chatting to...","[See you!, Have a nice day, Bye! Come back aga...",[]
2,thanks,"[Thanks, Thank you, That's helpful, Awesome, t...","[Happy to help!, Any time!, My pleasure]",[]
3,noanswer,[],"[Sorry, can't understand you, Please give me m...",[]
4,options,"[How you could help me?, What you can do?, Wha...",[I can guide you through Adverse drug reaction...,[]
5,adverse_drug,"[How to check Adverse drug reaction?, Open adv...",[Navigating to Adverse drug reaction module],[]
6,blood_pressure,"[Open blood pressure module, Task related to b...",[Navigating to Blood Pressure module],[]
7,blood_pressure_search,[I want to search for blood pressure result hi...,"[Please provide Patient ID, Patient ID?]",[search_blood_pressure_by_patient_id]
8,search_blood_pressure_by_patient_id,[],[Loading Blood pressure result for Patient],[]
9,pharmacy_search,"[Find me a pharmacy, Find pharmacy, List of ph...",[Please provide pharmacy name],[search_pharmacy_by_name]


2. Preprocess data

    - When working with text data, we need to perform various preprocessing on the data before we make a machine learning or a deep learning model. Based on the requirements we need to apply various operations to preprocess the data.

    - Tokenizing is the most basic and first thing you can do on text data. Tokenizing is the process of breaking the whole text into small parts like words.

    - Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [27]:
print(words)

["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [29]:
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

In [28]:
print(classes)

['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


Now we will lemmatize each word and remove duplicate words from the list. Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [6]:
# lemmatize, lower each word and remove duplicates
nltk.download('omw-1.4')
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\home_\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


3. Create training and testing data

    - Now, we will create the training data in which we will provide the input and the output. Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers.

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\home_\AppData\Local\Temp/ipykernel_16568/2748295590.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


4. Build the model

    - We have our training data ready, now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, we achieved 100% accuracy on our model. Let us save the model as ‘chatbot_model.h5’.

In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

C:\Users\home_\AppData\Roaming\Python\Python39\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
10/10 [==============================] - 3s 10ms/step - loss: 2.3095 - accuracy: 0.0851
Epoch 2/200
10/10 [==============================] - 0s 13ms/step - loss: 2.1768 - accuracy: 0.2340
Epoch 3/200
10/10 [==============================] - 0s 9ms/step - loss: 2.1894 - accuracy: 0.1489
Epoch 4/200
10/10 [==============================] - 0s 10ms/step - loss: 1.9935 - accuracy: 0.3617
Epoch 5/200
10/10 [==============================] - 0s 8ms/step - loss: 1.9207 - accuracy: 0.4255
Epoch 6/200
10/10 [==============================] - 0s 7ms/step - loss: 1.8233 - accuracy: 0.3404
Epoch 7/200
10/10 [==============================] - 0s 7ms/step - loss: 1.6594 - accuracy: 0.5319
Epoch 8/200
10/10 [==============================] - 0s 8ms/step - loss: 1.4549 - accuracy: 0.6383
Epoch 9/200
10/10 [==============================] - 0s 8ms/step - loss: 1.4158 - accuracy: 0.5106
Epoch 10/200
10/10 [==============================] - 0s 10ms/step - loss: 1.2197 - accuracy: 0.5957
Epoch

10/10 [==============================] - 0s 8ms/step - loss: 0.0558 - accuracy: 0.9787
Epoch 83/200
10/10 [==============================] - 0s 8ms/step - loss: 0.1094 - accuracy: 0.9787
Epoch 84/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0233 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0254 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0340 - accuracy: 0.9787
Epoch 87/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0285 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 8ms/step - loss: 0.1671 - accuracy: 0.9574
Epoch 89/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0372 - accuracy: 0.9787
Epoch 90/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0624 - accuracy: 0.9787
Epoch 91/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0945 - accuracy: 0.9787
Epoch 92/200


10/10 [==============================] - 0s 8ms/step - loss: 0.0096 - accuracy: 1.0000
Epoch 164/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0101 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0312 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0210 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0153 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0217 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0093 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 8ms/step - loss: 0.0172 - accuracy: 1.0000
Epoc

5. Predict the response (Graphical User Interface)

    - To predict the sentences and get a response from the user to let us create a new file ‘chatapp.py’.

    - We will load the trained model and then use a graphical user interface that will predict the response from the bot. The model will only tell us the class it belongs to, so we will implement some functions which will identify the class and then retrieve us a random response from the list of responses.

    - Again we import the necessary packages and load the ‘words.pkl’ and ‘classes.pkl’ pickle files which we have created when we trained our model:

In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('chatbot-python-project-data-codes/intents.json').read())
words = pickle.load(open('chatbot-python-project-data-codes/words.pkl','rb'))
classes = pickle.load(open('chatbot-python-project-data-codes/classes.pkl','rb'))

To predict the class, we will need to provide input in the same way as we did while training. So we will create some functions that will perform text preprocessing and then predict the class.

In [11]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

After predicting the class, we will get a random response from the list of intents.

In [12]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

Now we will develop a graphical user interface. Let’s use Tkinter library which is shipped with tons of useful libraries for GUI. We will take the input message from the user and then use the helper functions we have created to get the response from the bot and display it on the GUI. Here is the full source code for the GUI.

In [14]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

6. Run the chatbot

To run the chatbot, we have two main files; train_chatbot.py and chatapp.py.

First, we train the model using the command in the terminal:

`python train_chatbot.py`

If we don’t see any error during training, we have successfully created the model. Then to run the app, we run the second file.

`python chatgui.py`

The program will open up a GUI window within a few seconds. With the GUI you can easily chat with the bot.